##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Gemini进行笔记整理


In [6]:
%pip install -U -q "google-genai>=1.16.0"

In [7]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)
MODEL_ID="gemini-2.5-flash-preview-05-20" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro", "gemini-2.5-flash-preview-05-20", "gemini-2.5-pro-preview-06-05"] {"allow-input":true, isTemplate: true}
import json
from PIL import Image
from IPython.display import display, Markdown

上传txt文件，整理得到相应由音频整理所得笔记。

In [ ]:
# 文件读取
filename = r"/content/5660.txt"
with open(filename, 'r', encoding='utf-8') as f:
    text_content = f.read()

# 构造 prompt
prompt = f"""
请你将R这里的知识点进行整理，要求进行结构化编辑，放入一个md文件，每个单词下面有

meanings、usage、related word、notes共四条属性，word用一级标题，meaning如果有多个意思就标明词性，用bullet point呈现；usage也用bullet point呈现，每个usage的格式是"例子"（本单词在这个例子中的含意），这个例子可以是一个具体的句子，也可以是一个固定搭配；related word是它的衍生词、近义词、反义词等等有提到的相关词，如果它的relatedword有进一步解释，那么在这个单词下面列出它的related word，然后这个relatedword就另外用二级标题写出word后，剩下的四条属性按照相同规则编写，如果只是简单提一下relatedword，就在原单词下面简单描述一下即可。所有的phrase搭配、例句都分行列入usage里面；notes用来呈现词根词缀、记忆方法。

如果有提到这个属性，就用bulletpoint列出这个属性+：+低一级bulletpoint，如果没有则不提，如果无法归类到前三条属性，则归类为notes。

{text_content}
"""
thinking_budget = 4096 # @param {type:"slider", min:0, max:24576, step:1}

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_budget=thinking_budget
        )
    )
)

# 将 response.text 保存为 result.txt 文件
with open("result.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

print("保存完成：result.txt")

In [ ]:
import os
import re
input_dir = "/content/files"
output_file = "/content/result.md"

# 先清空result.md（如果已存在）
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("")
# 重新排序
def extract_number(filename):
    # 用正则匹配文件名中的数字部分
    match = re.search(r'(\d+)', filename)
    if match:
        return int(match.group(1))
    else:
        # 如果没有数字，返回一个大数，保证排序靠后
        return float('inf')
files = [f for f in os.listdir(input_dir) if f.endswith('.txt')]
files_sorted = sorted(files, key=extract_number)
thinking_budget = 4096 # @param {type:"slider", min:0, max:24576, step:1}
for filename in files_sorted:
    if not filename.endswith(".txt"):
        continue
    input_path = os.path.join(input_dir, filename)

    with open(input_path, 'r', encoding='utf-8') as f:
        text_content = f.read()
    prompt = f"""
    请你提取以下内容，并用 Markdown 格式输出：

    核心单词（用三级标题 ### 单词 标明）
    本单词的固定搭配

    本单词的用法

    本单词的例句

    衍生词

    衍生词的用法和例句

    要求：

    核心单词用三级标题标明，如 ### abide，每个单词只出现一次，位于该单词所有表达之前；

    其他表达以有序列表形式呈现，格式如下：

    markdown
    复制
    编辑
    1. "英文表达" (知识点单词 本句中涉及的意思或用法 中文翻译)
        * 例句（保持原文，不做修改）
    只提取英文表达，

    “知识点单词”、“本句中涉及的意思或用法”、“中文翻译”由你基于原文内容准确理解并用简洁中文表达，避免逐字照搬；

    例句紧跟在对应表达下面，用缩进的 * 标记单独一行展示，保持例句的英文原文和标点不变；

    删除所有笔记中的注释、解释、翻译等无关内容，只保留上述表达和例句；

    同一表达有多条例句时，每条例句均单独列出；

    输出顺序严格为：核心单词标题 → 固定搭配 → 用法 → 例句 → 衍生词 → 衍生词用法及例句；

    确保中文释义精准简洁，便于记忆与回想。

    示例输入（示范）：
    pgsql
    复制
    编辑
    # abound

    - Meaning: Exist in large numbers
      - Used as an intransitive verb to indicate abundance.
      - Example: "Stories about his travels abound" (关于他的游历的故事有很多).
    - Phrase: abound with
      - Means to be full of or rich in something.
      - Example: "The lake abounds with fish" (这个湖有很多鱼).
    示例输出：
    markdown
    复制
    编辑
    ### abound
    1. "abound" (动词 表示数量多)
        * Stories about his travels abound (关于他的游历的故事有很多).
    2. "abound with" (固定搭配 充满)
        * The lake abounds with fish (这个湖有很多鱼).

    {text_content}
    """
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(
                thinking_budget=thinking_budget
            )
        )
    )
  # 追加写入统一result.md文件，且用一级标题分割
    with open(output_file, 'a', encoding='utf-8') as fout:
        # 以文件名（去除扩展名）作为一级标题
        section_title = os.path.splitext(filename)[0]
        fout.write(f"# {section_title}\n\n")
        fout.write(response.text)
        fout.write("\n\n")
    print(f"{filename} 处理完成，结果追加写入 {output_file}")

print("保存完成：result.txt")

SR1.txt 处理完成，结果追加写入 /content/result.md
SR2.txt 处理完成，结果追加写入 /content/result.md
SR3.txt 处理完成，结果追加写入 /content/result.md
SR4.txt 处理完成，结果追加写入 /content/result.md
SR5.txt 处理完成，结果追加写入 /content/result.md
